## Creating Input-Target Pairs

In [3]:
import tiktoken

In [4]:
# Let's load the data

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Initialise the tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# encode the text data
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [5]:
CONTEXT_SIZE = 4    # length of the input to llm
# The context size (or context window) determines how many tokens the model can consider at once when predicting the next token.

x = enc_text[:CONTEXT_SIZE]
y = enc_text[1:CONTEXT_SIZE+1]

print(f"x: {x}")
print(f"y: {y}")

x: [40, 367, 2885, 1464]
y: [367, 2885, 1464, 1807]


In [6]:
# so let's create the next prediction task

for i in range(1, CONTEXT_SIZE+1):
    context = enc_text[:i]
    desired = enc_text[i]

    print(f"{context} ---------------> {desired}")

[40] ---------------> 367
[40, 367] ---------------> 2885
[40, 367, 2885] ---------------> 1464
[40, 367, 2885, 1464] ---------------> 1807


In [7]:
# Decode the ids to see the prediction with words or tokens
for i in range(1, CONTEXT_SIZE+1):
    context = enc_text[:i]
    desired = enc_text[i]


    print(f"{tokenizer.decode(context)} ------------> {tokenizer.decode([desired])}")

I ------------>  H
I H ------------> AD
I HAD ------------>  always
I HAD always ------------>  thought


## Implementing data loaders

We will implement a data loader that fetches input-output target pairs using a sliding window approach

- To implement efficient dataloaders, we collect inputs in a tensor `x`, where each row represents one input context. The second tensor `y` contains the corresponding prediction targets (next words), which are created by shifting the input by one position

- Step 1: tokenize the entire text 
- Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length
- Step 3: Return the total number of rows in the dataset
- Step 4: Return a single row from the dataset

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # tokenize the text
        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [9]:
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # create dataset
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)

    # create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_iter = next(data_iter)


print(first_iter)


[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [15]:
second_iter = next(data_iter)
print(second_iter)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]
